# Astropy and SunPy - A Quick Primer

Before we progress to searching for DKIST data we need to cover some functionality of the [SunPy](https://sunpy.org) and [Astropy](https://astropy.org) packages.
These two packages, amongst a few others, are the core components of the DKIST tools.
In this session we shall only quickly cover the functionality of SunPy and Astropy we need for the rest of the workshop.
There are many other parts of these packages which are useful when working with DKIST data, which you should explore.
As we covered in the introduction, Python is a modular language; Astropy provides fundamental functionality for a lot of different types of astronomy.
In this section we shall cover the following parts of Astropy:

* Units
* World Coordinate Systems
* Coordinates

## Units

Astropy provides a subpackage {obj}`astropy.units` which provides tools for associating physical units with numbers and arrays.
This lets you do mathematical operations on these arrays while propagating the units.

To get started we import `astropy.units` because we are going to be using this a lot, we import it as `u`.

Now we can access various units as such:

We can now attach a unit to a number:

We can also compose multiple quantities (a number with a unit):

Using the `.to()` method on a `u.Quantity` object lets you convert a quantity to a different unit.

### Equivalencies

Some conversions are not done by a conversion factor as between miles and kilometers – for example converting between wavelength and frequency:

However we can make use of a spectral *equivalency* to indicate the link between the units:

### Constants

The `astropy.constants` sub-package provides a set of physical constants which are compatible with the units/quantities framework:

## Coordinates

The Astropy coordinates submodule {obj}`astropy.coordinates` provides classes to represent physical coordinates with all their associated metadata, and transform them between different coordinate systems.
Currently, {obj}`astropy.coordinates` supports: 

* Spatial coordinates via {obj}`astropy.coordinates.SkyCoord`
* Spectral coordinates via {obj}`astropy.coordinates.SpectralCoord`
* Stokes profiles via {obj}`astropy.coordinates.StokesCoord` (coming soon)

### Spatial Coordinates

SunPy provides extensions to the Astropy coordinate system to represent common solar physics frames.
So to use the sunpy coordinates we have to first import {obj}`sunpy.coordinates` which registers them with astropy.

We can now create a `SkyCoord` object representing a point on the Sun:

This is the most minimal version of this coordinate frame, however, it isn't very useful as we haven't provided enough information to be able to transform it to other frames.
This is because helioprojective is an observer centred coordinate frame, so we need to know where in inertial space the observer is.
One way of doing this is to say the observer is on Earth at a specific time:

This coordinate can now be converted to other frames, such as heliographic coordinates:

There are few things to notice about the difference between these two `SkyCoord` objects:

1. The default representation of the latitude and longitude is now in degrees as is conventional.
1. The heliographic frame is three dimensional (it has a radius), when the input frame was not. This is because the distance from the observer was calculated using the `rsun` attribute.
1. The `obstime` and `rsun` attributes are still present, but the `observer` attribute isn't. This is because heliographic coordinates are not observer dependent.
1. The `obstime` attribute is still important to transform to other frames, as the heliographic frame needs to know the location of Earth.

### Spectral Coordinates

{obj}`astropy.coordinates.SpectralCoord` is a `Quantity` like object which also holds information about the observer and target coordinates and relative velocities.

```{note}
Use of `SpectralCoord` with solar data is still experimental so not all features may work, or be accurate.
```

`SpectralCoord` does not automatically make the HPC coordinate 3D, but wants the distance, so we do it explicitally:

Now we can construct our spectral coordinate with both a target and an observer

We can show the full details of the spectral coord (working around a bug in astropy):

## World Coordinate System

One of the other core components of the ecosystem provided by Astropy is the {obj}`astropy.wcs` package which provides tools for mapping pixel to world coordinates and world to pixel.
When loading a FITS file with complete (and standard compliant) WCS metadata we can create an `astropy.wcs.WCS` object.
For the this example we will use some sunpy sample data from AIA.

To read this FITS file we will use {obj}`astropy.io.fits` (you can also use `sunpy` for this).

We can now access the header of the second HDU:

Using this header we can create a `astropy.wcs.WCS` object:

This WCS object allows us to convert between pixel and world coordinates, for example:

This call returns a {obj}`astropy.coordinates.SkyCoord` object (which needs sunpy to be imported), we will come onto this more later.

We can also convert between pixel and plain numbers:

The units for these values are given by:

The WCS also has information about what the world axes represent:

We can also inspect the correlation between the world axes and pixel axes:

This correlation matrix has the world dimensions as rows, and the pixel dimensions as columns.
As we have a 2D image here, with two pixel and two world axes where both are coupled together.
This means that to calculate either latitude or longitude you need both pixel coordinates.